# Face detection

In [80]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [81]:
cap = cv2.VideoCapture('http://192.168.43.1:4747/mjpegfeed')
# cap = cv2.VideoCapture(1)
while True:
    ## Read the frame
    ok, img = cap.read()
    if not ok:
        print('Video Ending')
        cap.release()
        cv2.destroyAllWindows()
        break
    ## Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ## Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    ## Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    ## Display
    cv2.imshow('img', img)
    ## quit Press Key Q to quit and Close window
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Release the VideoCapture object
        cap.release()
        # Close all window
        cv2.destroyAllWindows()
        break


<ipython-input-81-cfe35b833b3d>:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if (faces !=()):


# Image to Face Encoder

In [99]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os
 
#get paths of each file in folder named Images
#Images here contains my data(folders of various persons)
imagePaths = list(paths.list_images('../Pictures/Ananth_/'))
knownEncodings = []
knownNames = []
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    name = imagePath.split(os.path.sep)[-1]
    # load the input image and convert it from BGR (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}
#use pickle to save data into a file for later use
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

In [8]:
from imutils import paths
imagePaths = list(paths.list_images('../Pictures/Ananth_/'))
for (i,imagepath) in enumerate(imagePaths):
    print(imagepath.split(os.path.sep)[-1])

ananth0.jpg
ananth2.jpg
kamal2.jpg
ananth1.jpg
kamal3.jpg
ananth4.jpg
yogesh.jpg
kamal0.jpg
ananth3.jpg
kamal1.jpg
Ananth.jpg


# Face Recognition using Photo

In [102]:
import face_recognition
import imutils
import pickle
import time
import cv2
import os
 
#find path of xml file containing haarcascade file
cascPathface = "haarcascade_frontalface_default.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())
#Find path to the image you want to detect face and pass it here
image = cv2.imread('../Pictures/Ananth_/navaneeth0.jpg' )

rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#convert image to Greyscale for haarcascade
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow("hi",gray)
# cv2.waitKey(15000)
# cv2.destroyAllWindows()
faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(60, 60),flags=cv2.CASCADE_SCALE_IMAGE)
print(faces)
 
# the facial embeddings for face in input
encodings = face_recognition.face_encodings(rgb)
names = []
# loop over the facial embeddings incase
# we have multiple embeddings for multiple fcaes
for encoding in encodings:
    #Compare encodings with encodings in data["encodings"]
    #Matches contain array with boolean values and True for the embeddings it matches closely
    #and False for rest
    matches = face_recognition.compare_faces(data["encodings"],encoding,tolerance=0.44)
    #set name =inknown if no encoding matches
    name = "Unknown"
    # check to see if we have found a match
    if True in matches:
        print("matched")
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            #Check the names at respective indexes we stored in matchedIdxs
            name = data["names"][i]
            #increase count for the name we got
            counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
 
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image

            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (255,255, 0), 2)
        # update the list of names
    names.append(name)
        # loop over the recognized faces
    for ((x, y, w, h), name) in zip(faces, names):
        # rescale the face coordinates
        # draw the predicted face name on the image

        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image, name[:-5], (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (255,255, 0), 2)
    cv2.imshow("Frame", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[[264  84 179 179]]
matched


In [ ]:

j=open('face_enc','rb')
print(pickle.load(j))


# Face Recognition Live Feed

In [197]:
import face_recognition
import imutils
import pickle
import time
import cv2
import os
 
#find path of xml file containing haarcascade file 
cascPathface = "haarcascade_frontalface_default.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())
 
print("Streaming started")
video_capture = cv2.VideoCapture('http://192.168.43.1:4747/mjpegfeed')
# video_capture = cv2.VideoCapture(1)
# loop over frames from the video file stream
names=[]
while True:
    # grab the frame from the threaded video stream
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=6,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
 
    # convert the input frame from BGR to RGB 
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # the facial embeddings for face in input
    encodings = face_recognition.face_encodings(rgb)
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple fcaes
    pseudo_names=[]
    for encoding in encodings:
       #Compare encodings with encodings in data["encodings"]
       #Matches contain array with boolean values and True for the embeddings it matches closely
       #and False for rest
        matches = face_recognition.compare_faces(data["encodings"],encoding,tolerance=0.44)
        #set name =inknown if no encoding matches
        name = "Unknown     "
        # check to see if we have found a match
        if True in matches:
            #Find positions at which we get True and store them
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                #Check the names at respective indexes we stored in matchedIdxs
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
        # update the list of names
            names.append(name)
            pseudo_names.append(name)
            

        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, pseudo_names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, name[:-5], (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1)&0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

Streaming started


# Attendance Marker

In [194]:
namesa = list({name[:-5] for name in names})
namesa

['ananth', 'veda']

In [178]:
import xlsxwriter
import time

date = time.strftime('%d-%m-%Y')
time = time.strftime('%H-%M-%S')

workbook = xlsxwriter.Workbook(str(date)+' CSE (AI & ML).xlsx')
worksheet = workbook.add_worksheet(str(date))

worksheet.write(0,0,'NAME')
worksheet.write(0,1,'TIME')

for i in range(len(namesa)):
    if namesa[i][0]!='U': 
        worksheet.write(i+1,0,namesa[i])
        worksheet.write(i+1,1,time)
    
workbook.close()

In [168]:
import pandas as pd
df=pd.read_excel("27-10-2021 CSE (AI & ML).xlsx")
df.empty

False